# INF391 - Tarea 9
El objetivo de esta tarea es construir un sistema de recomendación basado en contenido usando procesamiento del lenguaje natural.
- El conjunto de datos son las 250 películas mejor ranqueadas de IMDB.
- Las recomendaciones estarán basadas en información como directores, actores, género y descripción de la película.
- Realizar una limpieza de los datos para considerar solo las palabras más relevantes de la descripción de la película.
- Vectorizar cada película y calcular su similaridad con el resto.
- La entrada será el título de una película y la salida debe ser una lista con las 10 más similares (Top-10).
- ¿Cómo cambian las recomendaciones si están basadas únicamente en el título de la película?
- ¿Alguna otra *feature* del conjunto original sería interesante incluir en el análisis?

In [1]:
import pandas as pd
df = pd.read_csv('IMDB_Top250.csv')

In [2]:
df = df[['Title','Genre','Director','Actors','Plot']]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [3]:
# Librerías a utilizar
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

# Desarrollo

Para realizar la recomendación, se irá explicando el paso a paso para entender el desarrollo e ir cumpliendo con los objetivos esperados, además de entender las decisiones que se fueron tomando en las distintas etapas.

En primer lugar, se decidió utilizar por defecto la información del (o los) directos, actores, género y descripción de la trama, además de utilizar el mismo título de la película en el análisis al incluir términos que pueden ser relevantes en el contenido a analizar. Es por ello que se trabajó el dataframe, estandarizando a minúsculas, eliminando puntuación y stopwords conocidas en inglés, según la librería nltk.

In [4]:
# Limpieza dataframe
def limpiar_df(df):
    ## A lower case y eliminar puntuación
    ## (Dejar nombres capitalizados)
    df['Title-clean'] = df['Title'].str.lower().str.replace('[,.:]', '', regex=True)
    df['Genre'] = df['Genre'].str.lower().str.replace(',', '', regex=True)
    df['Plot'] = df['Plot'].str.lower().str.replace('[,.:]', '', regex=True)
    df['Actors'] = df['Actors'].str.replace('[,.:]', '', regex=True)
    df['Director'] = df['Director'].str.replace('[,.:]', '', regex=True)
    ## Eliminar stopwords de plot
    sw = stopwords.words('english')
    df['Title-clean'] = df['Title-clean'].apply(lambda x: ' '.join([palabra for palabra in x.split() if palabra not in sw]))
    df['Plot'] = df['Plot'].apply(lambda x: ' '.join([palabra for palabra in x.split() if palabra not in sw]))
    # Agregar columna ID
    df['id'] = df.index

Con el dataframe limpio, resulta necesario componer todo el contenido (de directores, descripción, título, etc.) en un solo objeto para poder realizar la comparación. Luego, dicho objeto se debe analizar usando nlp para estandarizar los términos ahí incluidos. Para ello se convirtió todo el contenido en lemmas.

In [5]:
# Unir contenido en un string
# lista de features
def componerContenido(df, features):
    df_contenido = pd.DataFrame(index=df.index)
    df_contenido['id'] = df['id']
    # Concatenar
    total = ""
    for feat in features:
        parcial = df[feat]
        total += " "+parcial
    df_contenido['contenido'] = total
    df_contenido['contenido'].str.strip()
    return df_contenido

# Proceso de lemmatización
lemmatizer = WordNetLemmatizer()
lematizar = lambda string: lemmatizer.lemmatize(string)

# Transformar lista en lista lematizada
def lemmatizarLista(lista):
    lista=list(map(lematizar,lista))
    return lista

# Transformar string con contenido completo en lemma
def lemmatizarContenido(df):
    df['lemma'] = df['contenido'].str.split().apply(lambda x: lemmatizarLista(x))

Luego, con el contenido lematizado, se debe vectorizar, calculando las frecuencias y el tf-idf para poder evaluar comparativamente el contenido. Después, con la matriz con el contenido de la película vectorizado, se calcula la similaridad coseno, para poder recomendar los más parecidos.

In [6]:
# Aplicar funciones antes definidas para pre procesar datos según los
# features deseados
def preProcess(df, features=['Title', 'Genre', 'Director', 'Actors', 'Plot']):
    limpiar_df(df)
    contenido = componerContenido(df, features)
    lemmatizarContenido(contenido)
    return contenido

# Vectorizar contenido y armar las matrices de frecuencias y tf-idf
def vectorizar(contenido):
    # Obtener cada palabra en el contenido, el cual incluye todas las películas
    palabras = sum(contenido['lemma'].tolist(), [])
    palabras = list(set(palabras))
    # Inicializar variables frecuencia, idf
    D = df.shape[0]
    n_palabras = len(palabras)
    frecuencias = np.zeros((n_palabras, D))
    idf = np.zeros((n_palabras))
    # Contar frecuencia apariciones en cada película
    # y películas que la incluyen para idf
    i = 0
    size = contenido.shape[0]
    for pal in palabras:
        D_actual = 0
        for j in range(size):
            freq = contenido['lemma'][j].count(pal)
            frecuencias[i][j] = freq
            if freq > 0:
                D_actual += 1
        idf[i] = np.log2(D/D_actual)
        i += 1
    # Con las frecuencias calcular tf
    tf = np.copy(frecuencias)
    i = 0
    for pal in palabras:
        for j in range(size):
            tf_j = frecuencias[i][j]
            tf_doc = np.sum(frecuencias[:][j])
            tf[i][j] = tf_j / tf_doc
        i += 1
    # Calcular tf-idf
    tf_idf = (tf.T * idf).T
    return frecuencias, tf_idf

# Cálculo similitud coseno vectores
def similitudCos(tf_idf):
    similitud = cosine_similarity(tf_idf.T)
    return similitud

Finalmente, ya con la matriz de películas vectorizadas se procede a recomendar las Top 10 dada una película en específico.

In [7]:
# Armar lista con las top N películas dado un título dentro del listado de las top 250 de imdb
def topN_peliculas(df_peliculas, pelicula, similitud, Ntop=10):
    size = similitud.shape[0]
    # Quitar diagonal (1 siempre es el más alto)
    similitud_sin_diag = similitud - np.eye(size)
    # Obtener id según el título de la película
    index_pelicula = df_peliculas[df_peliculas['Title'] == pelicula]['id'].item()
    top_peliculas = []
    scores = []
    for i in range(Ntop):
        # Calcular máximo
        max_score = similitud_sin_diag[index_pelicula].max()
        index_top = similitud_sin_diag[index_pelicula].argmax()
        # Quitar top de lista actual
        similitud_sin_diag[index_pelicula, index_top] = -1
        # Extraer pelicula top
        pelicula_top = df_peliculas[(df_peliculas['id'] == index_top)]['Title'].item()
        top_peliculas.append(pelicula_top)
        scores.append(max_score)
    return top_peliculas, scores

# Mostrar peliculas top con puntaje
def imprimirTopN(titulo_pelicula, top_peliculas, scores):
    print("\nPelícula \""+titulo_pelicula+"\"")
    i = 0
    while (i<10):
        print(str(i+1)+". \""+top_peliculas[i]+"\" - Score: "+str(np.round(scores[i], 3)))
        i+=1
    print("\n")

Así, realizamos el proceso completo y probamos con tres casos arbitrarios para poder evaluar los resultados. En este caso se escogieron: "The Godfather", "Inside Out" y "Pulp Fiction".


In [8]:
df_copia = df.copy()
contenido = preProcess(df_copia)
freqs, tf_idf = vectorizar(contenido)
similitud = similitudCos(tf_idf)

In [9]:
titulo = "The Godfather"
topN, scores = topN_peliculas(df_copia, pelicula=titulo, similitud=similitud)
imprimirTopN(titulo, topN, scores)


Película "The Godfather"
1. "The Godfather: Part II" - Score: 0.181
2. "Apocalypse Now" - Score: 0.145
3. "On the Waterfront" - Score: 0.073
4. "Scarface" - Score: 0.069
5. "Heat" - Score: 0.061
6. "The Night of the Hunter" - Score: 0.06
7. "A Streetcar Named Desire" - Score: 0.058
8. "Guardians of the Galaxy" - Score: 0.056
9. "Casino" - Score: 0.051
10. "Who's Afraid of Virginia Woolf?" - Score: 0.044




In [10]:
titulo = "Inside Out"
topN, scores = topN_peliculas(df_copia, pelicula=titulo, similitud=similitud)
imprimirTopN(titulo, topN, scores)


Película "Inside Out"
1. "Up" - Score: 0.07
2. "Monsters, Inc." - Score: 0.068
3. "Spider-Man: Homecoming" - Score: 0.062
4. "Vertigo" - Score: 0.059
5. "Harvey" - Score: 0.048
6. "L.A. Confidential" - Score: 0.046
7. "Out of the Past" - Score: 0.041
8. "Groundhog Day" - Score: 0.037
9. "Mr. Smith Goes to Washington" - Score: 0.035
10. "The Grapes of Wrath" - Score: 0.033




In [11]:
titulo = "Pulp Fiction"
topN, scores = topN_peliculas(df_copia, pelicula=titulo, similitud=similitud)
imprimirTopN(titulo, topN, scores)


Película "Pulp Fiction"
1. "Reservoir Dogs" - Score: 0.107
2. "The Shawshank Redemption" - Score: 0.106
3. "Inglourious Basterds" - Score: 0.091
4. "Some Like It Hot" - Score: 0.079
5. "Django Unchained" - Score: 0.067
6. "Goodfellas" - Score: 0.056
7. "Rope" - Score: 0.054
8. "Raging Bull" - Score: 0.05
9. "Sin City" - Score: 0.049
10. "A Beautiful Mind" - Score: 0.046




Así, para "El padrino" se obtiene lo esperado que la principal recomendación es la parte II, ya que en todo sentido son similares. En "Pulp Fiction" fue capaz de reconocer películas de temáticas parecidas y incluso recomendó muchas películas de Tarantino en el listado. Igualmente, para "Inside Out", las recomendaciones principalmente se basron en películas animadas y de contenido más ligero como comedias.

Para concluir, se puede ver que las recomendaciones hechas son capaces de inferir del contenido cosas importantes de las películas, así como sus actores, directores, temática o el tipo de película en cuestión. Las recomendaciones además, pueden incluir otras películas que a simple vista no parecerían tener tanta relación.

- ¿Cómo cambian las recomendaciones si están basadas únicamente en el título de la película?
- ¿Alguna otra *feature* del conjunto original sería interesante incluir en el análisis?